In [ ]:
!pip install -q nnAudio

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from scipy.fftpack import fft, dct, fftshift
import os
from tqdm import tqdm
import torch.nn.functional as F
from scipy import stats 
from scipy import signal
from scipy.signal import spectrogram,butter

from nnAudio.Spectrogram import CQT1992v2

In [ ]:
frame = pd.read_csv('/kaggle/input/g2net-gravitational-wave-detection/training_labels.csv')

In [ ]:
i=5
idx = frame.iloc[i]['id']
print(frame.iloc[i]['target'])
directory = os.path.join('/kaggle/input/g2net-gravitational-wave-detection/','train', idx[0],idx[1],idx[2],idx+'.npy')
# print(frame)
sample = np.load(directory)
filte = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)
waves = np.hstack(sample)
waves = waves / np.max(waves)
waves = torch.from_numpy(waves).float()
z= filte(waves)
print(z.shape)
plt.imshow(z.squeeze())
# print(Sxx.shape)
# plt.plot(Sxx)
# print(z)

In [ ]:
class GWavesDataSet(Dataset):

    def __init__(self, csv_file, root_dir = os.path.join('/kaggle/input/g2net-gravitational-wave-detection/','train')):

        self.gwaves = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.mean = np.load('../input/meanstd-of-g2n/means.npy')
        self.std = np.load('../input/meanstd-of-g2n/std.npy')
        self.spectrogram = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)
    def __len__(self):
        return len(self.gwaves)

    def __getitem__(self, idx):
        
        file_id = frame.iloc[idx]['id']
        wave_forms = np.load(os.path.join(self.root_dir, file_id[0], file_id[1], file_id[2],file_id+'.npy')).astype('float32')
        waves = np.hstack(wave_forms)
        waves = waves / np.max(waves)
        waves = torch.from_numpy(waves).float()
        wave_forms = self.spectrogram(waves)
        target = int(frame.iloc[idx]['target'])

        sample  = {'waveforms': wave_forms, 'target':torch.FloatTensor([target])}
        return sample

In [ ]:
dataset = GWavesDataSet('/kaggle/input/g2net-gravitational-wave-detection/training_labels.csv')
# dataset[154828]['waveforms']

In [ ]:
def torch_nanmean(x):
    num = torch.where(torch.isnan(x), torch.full_like(x, 0), torch.full_like(x, 1)).sum()
    value = torch.where(torch.isnan(x), torch.full_like(x, 0), x).sum()
    return value / num

def accuracy(output, target):
    """Computes the accuracy for multiple binary predictions"""
    pred = output >= 0.5
    truth = target >= 0.5
    acc = pred.eq(truth).sum() / target.numel()
    return acc


In [ ]:
class G2N(nn.Module):
    def __init__(self):
        super(G2N,self).__init__()
    
        
        self.conv1 = nn.Conv2d(1, 32, 3,1,1)
        self.conv1_bn = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 32, 3,1,1)
        self.conv2_bn = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, 3,1,1)
        self.conv3_bn = nn.BatchNorm2d(64)
        
        self.conv4 = nn.Conv2d(64, 64, 3,1,1)
        self.conv4_bn = nn.BatchNorm2d(64)
        
        self.conv4 = nn.Conv2d(64, 64, 3,1,1)
        self.conv4_bn = nn.BatchNorm2d(64)
        
        self.ff1 = nn.Linear(69*193*64,512)
        self.ff1_bn = nn.BatchNorm1d(num_features=512)
        self.ff2 = nn.Linear(512,1024)
        self.ff2_bn = nn.BatchNorm1d(num_features=1024)
        self.ff3 = nn.Linear(1024,2048)
        self.ff3_bn = nn.BatchNorm1d(num_features=2048)
        self.ff4 = nn.Linear(2048,512)
        self.ff4_bn = nn.BatchNorm1d(num_features=512)
        self.ff5 = nn.Linear(512,32)
        self.ff5_bn = nn.BatchNorm1d(num_features=32)
        self.ff6 = nn.Linear(32,1)
        

        
    def forward(self, sample):
        x = F.relu(self.conv1_bn(self.conv1(sample)))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3(x)))
        x = F.relu(self.conv4_bn(self.conv4(x)))
        
        x = x.reshape(-1,64*193*69)
                   
        x = F.relu(self.ff1_bn(self.ff1(x)))
        x = F.relu(self.ff2(x))
        x = F.relu(self.ff3(x))
        x = F.relu(self.ff4_bn(self.ff4(x)))
        x = F.relu(self.ff5(x))
        x = torch.sigmoid(self.ff6(x))
        
        return x        

In [ ]:
# model = G2N()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                          shuffle=True, num_workers=8)
# len(dataloader)
# z = list()
# for data in tqdm(dataloader):
#     z.append(data['waveforms'])
# d = torch.stack(z)
# d = d.reshape(35000*16,300)
# mean = torch.mean(d,1).numpy()
# std = torch.std(d,1).numpy()
# print(mean)
# print(std)
# np.save('means.npy',mean)
# np.save('std.npy',std)

In [ ]:
import torch.optim as optim
model = G2N().cuda()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
# model.load_state_dict(torch.load('check_point_1.pth'))
model.train()
for epoch in range(50):  # loop over the dataset multiple times
    torch.save(model.state_dict(),'check_point.pth')
    progrss_loader = tqdm(dataloader)
    running_loss = 0.0
    for i, data in enumerate(progrss_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['waveforms'].cuda(), data['target'].cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        labels = labels
#         print(labels.shape)

#         print(outputs.shape)
        loss = criterion(outputs, labels)
        acc = accuracy(outputs.squeeze(1), labels.squeeze(1))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i+1) % 4 == 0:    # print every 2000 mini-batches
            progrss_loader.set_description('[accuracy: %d, epoch: %5d] loss: %.3f' %
                  (acc*100, epoch+1, running_loss / 4))
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 4))
            running_loss = 0.0
print('Finished Training')

In [ ]:
print(model)

In [ ]:
torch.cuda.empty_cache()